# Investing.com 크롤러 만들기

동생이 [Investing.com](https://www.investing.com/equities/) 에서 S&P 500 기업들의 정보를 크롤링 하는 것을 만들고 있길래 도와 줄 겸 나도 한번 해 보았다.

## 하고자 하는 것
1.   [S&P 500 기업 리스트](https://kr.investing.com/equities/StocksFilter?noconstruct=1&smlID=595&sid=&tabletype=price&index_id=166)를 받아온다
2.   각 기업 별 정보 페이지에서 원하는 정보를 추출해낸다.



## 사전 작업
### Library Import
일단 크롤링을 위해 requests 및 BeautifulSoup을 쓸 것이다. 두 개를 Import 하자

In [1]:
import requests
from bs4 import BeautifulSoup

### User Agent 설정
investing.com 은 하나의 User Agent에서 반복하여 요청을 하면 올바르지 않는 접근으로 간주하여 접속을 거부한다. 이를 위해 User Agent 리스트를 정의하고, 매 요청마다 임의의 User Agent를 선택하여 요청하자.

In [2]:
import random
USER_AGENTS = [
    "Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; en-US; rv:1.9.1b3) Gecko/20090305 Firefox/3.1b3 GTB5",
    "Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; ko; rv:1.9.1b2) Gecko/20081201 Firefox/3.1b2",
    "Mozilla/5.0 (X11; U; SunOS sun4u; en-US; rv:1.9b5) Gecko/2008032620 Firefox/3.0b5",
    "Mozilla/5.0 (X11; U; Linux x86_64; en-US; rv:1.8.1.12) Gecko/20080214 Firefox/2.0.0.12",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; cs; rv:1.9.0.8) Gecko/2009032609 Firefox/3.0.8",
    "Mozilla/5.0 (X11; U; OpenBSD i386; en-US; rv:1.8.0.5) Gecko/20060819 Firefox/1.5.0.5",
    "Mozilla/5.0 (Windows; U; Windows NT 5.0; es-ES; rv:1.8.0.3) Gecko/20060426 Firefox/1.5.0.3",
    "Mozilla/5.0 (Windows; U; WinNT4.0; en-US; rv:1.7.9) Gecko/20050711 Firefox/1.0.5",
    "Mozilla/5.0 (Windows; Windows NT 6.1; rv:2.0b2) Gecko/20100720 Firefox/4.0b2",
    "Mozilla/5.0 (X11; Linux x86_64; rv:2.0b4) Gecko/20100818 Firefox/4.0b4",
    "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.2) Gecko/20100308 Ubuntu/10.04 (lucid) Firefox/3.6 GTB7.1",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:2.0b7) Gecko/20101111 Firefox/4.0b7",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:2.0b8pre) Gecko/20101114 Firefox/4.0b8pre",
    "Mozilla/5.0 (X11; Linux x86_64; rv:2.0b9pre) Gecko/20110111 Firefox/4.0b9pre",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:2.0b9pre) Gecko/20101228 Firefox/4.0b9pre",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:2.2a1pre) Gecko/20110324 Firefox/4.2a1pre",
    "Mozilla/5.0 (X11; U; Linux amd64; rv:5.0) Gecko/20100101 Firefox/5.0 (Debian)",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:6.0a2) Gecko/20110613 Firefox/6.0a2",
    "Mozilla/5.0 (X11; Linux i686 on x86_64; rv:12.0) Gecko/20100101 Firefox/12.0",
    "Mozilla/5.0 (Windows NT 6.1; rv:15.0) Gecko/20120716 Firefox/15.0a2",
    "Mozilla/5.0 (X11; Ubuntu; Linux armv7l; rv:17.0) Gecko/20100101 Firefox/17.0",
    "Mozilla/5.0 (Windows NT 6.1; rv:21.0) Gecko/20130328 Firefox/21.0",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:22.0) Gecko/20130328 Firefox/22.0",
    "Mozilla/5.0 (Windows NT 5.1; rv:25.0) Gecko/20100101 Firefox/25.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.8; rv:25.0) Gecko/20100101 Firefox/25.0",
    "Mozilla/5.0 (Windows NT 6.1; rv:28.0) Gecko/20100101 Firefox/28.0",
    "Mozilla/5.0 (X11; Linux i686; rv:30.0) Gecko/20100101 Firefox/30.0",
    "Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20100101 Firefox/31.0",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:40.0) Gecko/20100101 Firefox/40.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:58.0) Gecko/20100101 Firefox/58.0",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/525.19 (KHTML, like Gecko) Chrome/1.0.154.53 Safari/525.19",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/525.19 (KHTML, like Gecko) Chrome/1.0.154.36 Safari/525.19",
    "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.10 (KHTML, like Gecko) Chrome/7.0.540.0 Safari/534.10",
    "Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/534.4 (KHTML, like Gecko) Chrome/6.0.481.0 Safari/534.4",
    "Mozilla/5.0 (Macintosh; U; Intel Mac OS X; en-US) AppleWebKit/533.4 (KHTML, like Gecko) Chrome/5.0.375.86 Safari/533.4",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/532.2 (KHTML, like Gecko) Chrome/4.0.223.3 Safari/532.2",
    "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.201.1 Safari/532.0",
    "Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/3.0.195.27 Safari/532.0",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/530.5 (KHTML, like Gecko) Chrome/2.0.173.1 Safari/530.5",
    "Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/534.10 (KHTML, like Gecko) Chrome/8.0.558.0 Safari/534.10",
    "Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/540.0 (KHTML,like Gecko) Chrome/9.1.0.0 Safari/540.0",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/9.0.600.0 Safari/534.14",
    "Mozilla/5.0 (X11; U; Windows NT 6; en-US) AppleWebKit/534.12 (KHTML, like Gecko) Chrome/9.0.587.0 Safari/534.12",
    "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.13 (KHTML, like Gecko) Chrome/9.0.597.0 Safari/534.13",
    "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.11 Safari/534.16",
    "Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US) AppleWebKit/534.20 (KHTML, like Gecko) Chrome/11.0.672.2 Safari/534.20",
    "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/14.0.792.0 Safari/535.1",
    "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.2 (KHTML, like Gecko) Chrome/15.0.872.0 Safari/535.2",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.7 (KHTML, like Gecko) Chrome/16.0.912.36 Safari/535.7",
    "Mozilla/5.0 (Windows NT 6.0; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.66 Safari/535.11",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8) AppleWebKit/535.19 (KHTML, like Gecko) Chrome/18.0.1025.45 Safari/535.19",
    "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
    "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
    "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.15 (KHTML, like Gecko) Chrome/24.0.1295.0 Safari/537.15",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1467.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/30.0.1599.101 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.1623.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.116 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/40.0.2214.38 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.71 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.62 Safari/537.36"
]
def random_user_agent():
    return str(random.choice(USER_AGENTS))

*random_user_agent* 함수는 USER_AGENTS 리스트에서 임의의 user agent를 선택하는 함수이다. USER_AGENT 함수는 [investpy](https://pypi.org/project/investpy/) 라이브러리에서 가져왔다.

## S&P 500 기업 리스트 추출
일단 S&P 500 기업 리스트 및 각 기업별 정보를 보여주는 페이지들의 HTML 소스를 모두 읽어오자. 

In [3]:
import re
def get_snp500_link_list():
  # 크롤링 할 페이지 주소
  url_snp500list = "https://kr.investing.com/equities/StocksFilter?noconstruct=1&smlID=595&sid=&tabletype=price&index_id=166"

  # Request Header 설정. User agent는 매번 임의로 선택한다.
  head = {
      "User-Agent": random_user_agent(),
      "X-Requested-With": "XMLHttpRequest",
      "Accept": "text/html",
      "Accept-Encoding": "gzip, deflate, br",
      "Connection": "keep-alive",
  }

  # HTML 정보를 읽어온다.
  url = url_snp500list
  response = requests.get(url, headers = head)
  soup = BeautifulSoup(response.content, "html.parser")

  # HTML을 파싱하여 S&P 500 리스트의 각 항목들을 추출한다.
  items = soup.select("#cross_rate_markets_stocks_1 tbody")[0].find_all("tr")

  # S&P 500 기업들의 정보 페이들에 대한 URL 및 기업 이름 추출하여 result 리스트에 저장한다.
  result = []

  base_url = "https://kr.investing.com"
  pattern = "equities\/.+$"
  for item in items:
    company = {}
    link = item.find_all("td")[1].find_all("a")[0]
    # 기업 정보페이지의 URL에서 정규식으로 기업 이름에 대한 부분만 추출해낸다.
    # 예를 들어, 보잉 사의 주소인 'https://kr.investing.com/equities/boeing-co' 에서 
    # boeing-co 부분만 추출해 낸  후
    companyname=re.findall(pattern,link.attrs['href'])[0][9:]
    # 끝에 -finalcial-summary 라는 텍스트를 붙여 원하는 주소를 완성한다.
    link = base_url + link.attrs['href'] + "-financial-summary"
    company["companyname"] = companyname
    company["link"] = link    
    result.append(company)
  return result

In [ ]:
snp500_link_list = get_snp500_link_list()

In [ ]:
print(snp500_link_list)

[{'companyname': 'boeing-co', 'link': 'https://kr.investing.com/equities/boeing-co-financial-summary'}, {'companyname': 'gen-motors', 'link': 'https://kr.investing.com/equities/gen-motors-financial-summary'}, {'companyname': 'chevron', 'link': 'https://kr.investing.com/equities/chevron-financial-summary'}, {'companyname': 'citigroup', 'link': 'https://kr.investing.com/equities/citigroup-financial-summary'}, {'companyname': 'bank-of-america', 'link': 'https://kr.investing.com/equities/bank-of-america-financial-summary'}, {'companyname': 'at-t', 'link': 'https://kr.investing.com/equities/at-t-financial-summary'}, {'companyname': 'caterpillar', 'link': 'https://kr.investing.com/equities/caterpillar-financial-summary'}, {'companyname': 'intel-corp', 'link': 'https://kr.investing.com/equities/intel-corp-financial-summary'}, {'companyname': 'microsoft-corp', 'link': 'https://kr.investing.com/equities/microsoft-corp-financial-summary'}, {'companyname': 'ford-motor-co', 'link': 'https://kr.inv

## 각 기업 별 HTML 내용들을 파일로 저장하기
파싱 작업은 시행착오가 많이 들기 때문에 오류가 날 때 마다 수정 된 내용을 테스트 하기 위해 사이트에 접속하는것은 비효율적이다. 이 때문에 일단 원하는 페이지들을 모두 파일로 저장하자.

### 구글 드라이브 설정
여기서는 Google Colaboratory을 이용하고 있기 때문에 파일로 저장한 내용들을 구글 드라이브에 저장할 것이다. 우선 구글 드라이브를 마운트하자.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')  

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
# 구글 드라이브는 /content/gdrive/My Drive에 마운트 되었다.
# snp500_page라는 폴더를 만들자.
page_save_dir = '/content/gdrive/My Drive/snp500_page'
os.mkdir(page_save_dir)

In [ ]:
# 각 회사 정보 페이지의 html을 파일로 저장
def save_company_page(company,number = None):
  # 파일 정렬을 편하게 하기 위해 파일 명 앞에 번호를 붙일 수 있도록 정의
  if number != None:
    filename = page_save_dir + "/" + str(number) + "_" + company["companyname"] + ".txt"
  else:
    filename = page_save_dir + "/" + company["companyname"] + ".txt"
    
  # 파일을 연 후
  with open(filename, "w") as file:
    result = {}
    # 역시 임의의 user_agent로 요청을 날리도록 헤더를 설정하고
    head = {
        "User-Agent": random_user_agent(),
        "X-Requested-With": "XMLHttpRequest",
        "Accept": "text/html",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
    }
    
    # 요청에 대한 Response를 모두 파일로 저장한다.
    response = requests.get(company["link"], headers = head)
    file.write(response.text)

In [ ]:
import time
pausetime = 3 # 3 초
# 읽어들여온 S&P 500 리스트에 대한 URL에 접속하여 HTML 파일을 저장한다.
for idx in range(len(snp500_link_list)):
  company = snp500_link_list[idx]
  save_company_page(company,idx)
  # 너무 빨리 요청을 하면 접속 거부가 일어나기 때문에 pausetime 만큼 잠시 멈춘 후 요청을 한다. 여기서는 3초정도 딜레이를 주었다.
  time.sleep(pausetime)

## 각 페이지 파싱
이제 각 기업 별 정보 페이지들에 대한 HTML 파일을 모두 파이로 저장했으니 각 파일들을 파싱하여 저장하자.
참고로, [린드](https://kr.investing.com/equities/linde-plc?cid=942017) 라는 기업은 주소체계가 특이하게 되어 있다. https://kr.investing.com/equities/linde-plc?cid=942017 처럼 끝에 ?cid=942017이 붙어 있어 이 기업은 그냥 수동으로 HTML 파일을 긁어 왔다.

In [6]:
#파일을 읽어 파싱한다.
def get_company_info_from_file(file):
  result = {}
  # HTML 파일을 연다.
  with open(file, 'r') as f:
    contents = f.read()  
    soup = BeautifulSoup(contents, "html.parser")

    header_tag = soup.select("#leftColumn div h1")[0]
    company_name = header_tag.get_text().strip()
    pattern = r"\(.*\)"
    symbol = re.findall(pattern,company_name)[0][1:-1]
    result['company_name'] = company_name 
    result['symbol'] = symbol

    # 원하는 정보가 있는 테이블에 대해 원하는 형태의 dict을 만들어 낸다.
    info_tags = soup.select("#rsdiv")[0].find_all(attrs={'class':'genTbl openTbl companyFinancialSummaryTbl'})[0].select("tbody tr")  
    for info in info_tags:
      contents = info.find_all("td")
      name=contents[0].get_text()
      contents = contents[1:]
      data = []
      for content in contents:
        data.append(content.get_text())
      if name == "총수익":
        result["total_income"] = data
      elif name == "총 이익":
        result["total_margine"] = data
      elif name == "영업 이익":
        result["salse_margin"] = data
      elif name == "순이익":
        result["pure_margin"] = data 
  return result

In [ ]:
result = []
# HTML 파일이 저장된 폴더의 모든 파일들을 순환하며 정보를 얻어낸 후 result 리스트에 넣는다.
for root, dirs, files in os.walk(page_save_dir):
  for file in files:
    filepath = page_save_dir + "/" + file
    info = get_company_info_from_file(filepath)
    result.append(info)

In [ ]:
for data in result:
  print(data)

{'company_name': 'Boeing Co (BA)', 'symbol': 'BA', 'total_income': ['11807', '16908', '20560', '19980'], 'total_margine': ['-1171', '140', '1852', '3050'], 'salse_margin': ['-2964', '-1353', '-2204', '1259'], 'pure_margin': ['-2376', '-628', '-1010', '1167']}
{'company_name': '제너럴 모터스 (GM)', 'symbol': 'GM', 'total_income': ['16778', '32709', '30826', '35473'], 'total_margine': ['996', '2627', '1728', '4312'], 'salse_margin': ['-1214', '657', '-554', '2304'], 'pure_margin': ['-758', '294', '-194', '2351']}
{'company_name': '셰브론 (CVX)', 'symbol': 'CVX', 'total_income': ['16142', '29244', '34676', '34724'], 'total_margine': ['7998', '13739', '14984', '14849'], 'salse_margin': ['-7595', '2983', '-8565', '3147'], 'pure_margin': ['-8270', '3599', '-6610', '2580']}
{'company_name': '씨티그룹 (C)', 'symbol': 'C', 'total_income': ['20119', '23283', '26378', '24926'], 'pure_margin': ['3230', '1316', '2522', '4979']}
{'company_name': '뱅크오브아메리카 (BAC)', 'symbol': 'BAC', 'total_income': ['21693', '24018